In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np

In [5]:
file_path = '/content/drive/MyDrive/ecommerce/2019-Oct.parquet'

In [4]:
df = pd.read_parquet(file_path)

In [5]:
df['cart'] = np.where(df['event_type'] == 'cart', 1, 0)
df['view'] = np.where(df['event_type'] == 'view', 1, 0)
df['purchased'] = np.where(df['event_type'] == 'purchase', 1, 0)

In [6]:
df = df[['event_type', 'user_id', 'product_id', 'category_code', 'brand', 'price', 'view', 'cart', 'purchased']]

In [7]:
bins = list(range(0, 3000, 300))

bins_label = [x / 300 for x in bins]
df['price_label'] = pd.cut(df['price'], bins, right = False, labels = bins_label[: -1])

In [8]:
df_product = df.groupby('product_id')['purchased', 'cart', 'view'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [9]:
df_product['rating_value'] = (df_product['purchased'] + df_product['cart'] * 0.7 + df_product['view'] * 0.1) / (df_product['purchased'] + df_product['cart'] + df_product['view'])

In [11]:
df_rating = df_product['rating_value']

In [12]:
df_event_rating = pd.merge(df, df_rating, on = 'product_id', how = 'outer')

In [13]:
df_event_rating = df_event_rating[['event_type', 'user_id', 'product_id', 'price_label', 'rating_value']]

In [14]:
df_pl0 = df_event_rating[df_event_rating['price_label'] == 0]
df_pl1 = df_event_rating[df_event_rating['price_label'] == 1]
df_pl2 = df_event_rating[df_event_rating['price_label'] == 2]
df_pl3 = df_event_rating[df_event_rating['price_label'] == 3]
df_pl4 = df_event_rating[df_event_rating['price_label'] == 4]
df_pl5 = df_event_rating[df_event_rating['price_label'] == 5]
df_pl6 = df_event_rating[df_event_rating['price_label'] == 6]
df_pl7 = df_event_rating[df_event_rating['price_label'] == 7]
df_pl8 = df_event_rating[df_event_rating['price_label'] == 8]

In [15]:
pl0 = round(df_pl0[df_pl0['event_type'] == 'purchase'].shape[0] / df_pl0.shape[0], 4) * 100
pl1 = round(df_pl1[df_pl1['event_type'] == 'purchase'].shape[0] / df_pl1.shape[0], 4) * 100
pl2 = round(df_pl2[df_pl2['event_type'] == 'purchase'].shape[0] / df_pl2.shape[0], 4) * 100
pl3 = round(df_pl3[df_pl3['event_type'] == 'purchase'].shape[0] / df_pl3.shape[0], 4) * 100
pl4 = round(df_pl4[df_pl4['event_type'] == 'purchase'].shape[0] / df_pl4.shape[0], 4) * 100
pl5 = round(df_pl5[df_pl5['event_type'] == 'purchase'].shape[0] / df_pl5.shape[0], 4) * 100
pl6 = round(df_pl6[df_pl6['event_type'] == 'purchase'].shape[0] / df_pl6.shape[0], 4) * 100
pl7 = round(df_pl7[df_pl7['event_type'] == 'purchase'].shape[0] / df_pl7.shape[0], 4) * 100
pl8 = round(df_pl8[df_pl8['event_type'] == 'purchase'].shape[0] / df_pl8.shape[0], 4) * 100

In [29]:
df_event_rating['price_label'] = df_event_rating['price_label'].replace(0, pl0)
df_event_rating['price_label'] = df_event_rating['price_label'].replace(1, pl1)
df_event_rating['price_label'] = df_event_rating['price_label'].replace(2, pl2)
df_event_rating['price_label'] = df_event_rating['price_label'].replace(3, pl3)
df_event_rating['price_label'] = df_event_rating['price_label'].replace(4, pl4)
df_event_rating['price_label'] = df_event_rating['price_label'].replace(5, pl5)
df_event_rating['price_label'] = df_event_rating['price_label'].replace(6, pl6)
df_event_rating['price_label'] = df_event_rating['price_label'].replace(7, pl7)
df_event_rating['price_label'] = df_event_rating['price_label'].replace(8, pl8)

In [36]:
df_event_rating['price_label'] = df_event_rating['price_label'].astype(float)

In [37]:
df_event_rating['event_type'] = df_event_rating['event_type'].replace('view', 0.1)
df_event_rating['event_type'] = df_event_rating['event_type'].replace('cart', 0.7)
df_event_rating['event_type'] = df_event_rating['event_type'].replace('purchase', 1)

In [38]:
df_event_rating['event'] = pd.to_numeric(df_event_rating['event_type'])

In [39]:
df_event_rating.dtypes

event_type      float64
user_id           int64
product_id        int64
price_label     float64
rating_value    float64
event           float64
dtype: object

In [40]:
df_event_rating['rating'] = df_event_rating['price_label'] * df_event_rating['rating_value'] * df_event_rating['event_type']

In [41]:
df_user_product = df_event_rating[['user_id', 'product_id', 'rating']]

In [43]:
df_user_product

,user_id,product_id,rating
0,541312140,44600062,0.018381
1,541312140,44600062,0.018381
2,520558810,44600062,0.018381
3,520558810,44600062,0.018381
4,546203331,44600062,0.018381
...,...,...,...
42448759,563587006,49300114,0.020300
42448760,515624552,46800037,0.017300
42448761,515624552,46800037,0.017300
42448762,515474976,19002087,0.017300


In [4]:
def user_product(fpath):
  df = pd.read_parquet(fpath)

  df['cart'] = np.where(df['event_type'] == 'cart', 1, 0)
  df['view'] = np.where(df['event_type'] == 'view', 1, 0)
  df['purchased'] = np.where(df['event_type'] == 'purchase', 1, 0)

  df = df[['event_type', 'user_id', 'product_id', 'price', 'view', 'cart', 'purchased']]

  bins = list(range(0, 3000, 300))
  bins_label = [x / 300 for x in bins]
  df['price_label'] = pd.cut(df['price'], bins, right = False, labels = bins_label[: -1])

  df_product = df.groupby('product_id')['purchased', 'cart', 'view'].sum()
  df_product['rating_value'] = (df_product['purchased'] + df_product['cart'] * 0.7 + df_product['view'] * 0.1) / (df_product['purchased'] + df_product['cart'] + df_product['view'])

  df_rating = df_product['rating_value']
  df_event_rating = pd.merge(df, df_rating, on = 'product_id', how = 'outer')
  df_event_rating = df_event_rating[['event_type', 'user_id', 'product_id', 'price_label', 'rating_value']]

  df_pl0 = df_event_rating[df_event_rating['price_label'] == 0]
  df_pl1 = df_event_rating[df_event_rating['price_label'] == 1]
  df_pl2 = df_event_rating[df_event_rating['price_label'] == 2]
  df_pl3 = df_event_rating[df_event_rating['price_label'] == 3]
  df_pl4 = df_event_rating[df_event_rating['price_label'] == 4]
  df_pl5 = df_event_rating[df_event_rating['price_label'] == 5]
  df_pl6 = df_event_rating[df_event_rating['price_label'] == 6]
  df_pl7 = df_event_rating[df_event_rating['price_label'] == 7]
  df_pl8 = df_event_rating[df_event_rating['price_label'] == 8]

  pl0 = round(df_pl0[df_pl0['event_type'] == 'purchase'].shape[0] / df_pl0.shape[0], 4) * 100
  pl1 = round(df_pl1[df_pl1['event_type'] == 'purchase'].shape[0] / df_pl1.shape[0], 4) * 100
  pl2 = round(df_pl2[df_pl2['event_type'] == 'purchase'].shape[0] / df_pl2.shape[0], 4) * 100
  pl3 = round(df_pl3[df_pl3['event_type'] == 'purchase'].shape[0] / df_pl3.shape[0], 4) * 100
  pl4 = round(df_pl4[df_pl4['event_type'] == 'purchase'].shape[0] / df_pl4.shape[0], 4) * 100
  pl5 = round(df_pl5[df_pl5['event_type'] == 'purchase'].shape[0] / df_pl5.shape[0], 4) * 100
  pl6 = round(df_pl6[df_pl6['event_type'] == 'purchase'].shape[0] / df_pl6.shape[0], 4) * 100
  pl7 = round(df_pl7[df_pl7['event_type'] == 'purchase'].shape[0] / df_pl7.shape[0], 4) * 100
  pl8 = round(df_pl8[df_pl8['event_type'] == 'purchase'].shape[0] / df_pl8.shape[0], 4) * 100

  df_event_rating['price_label'] = df_event_rating['price_label'].replace(0, pl0)
  df_event_rating['price_label'] = df_event_rating['price_label'].replace(1, pl1)
  df_event_rating['price_label'] = df_event_rating['price_label'].replace(2, pl2)
  df_event_rating['price_label'] = df_event_rating['price_label'].replace(3, pl3)
  df_event_rating['price_label'] = df_event_rating['price_label'].replace(4, pl4)
  df_event_rating['price_label'] = df_event_rating['price_label'].replace(5, pl5)
  df_event_rating['price_label'] = df_event_rating['price_label'].replace(6, pl6)
  df_event_rating['price_label'] = df_event_rating['price_label'].replace(7, pl7)
  df_event_rating['price_label'] = df_event_rating['price_label'].replace(8, pl8)

  df_event_rating['price_label'] = df_event_rating['price_label'].astype(float)

  df_event_rating['event_type'] = df_event_rating['event_type'].replace('view', 0.1)
  df_event_rating['event_type'] = df_event_rating['event_type'].replace('cart', 0.7)
  df_event_rating['event_type'] = df_event_rating['event_type'].replace('purchase', 1)

  df_event_rating['event'] = pd.to_numeric(df_event_rating['event_type'])

  df_event_rating['rating'] = df_event_rating['price_label'] * df_event_rating['rating_value'] * df_event_rating['event_type']

  df_user_product = df_event_rating[['user_id', 'product_id', 'rating']]

  return df_user_product

In [6]:
df = user_product(file_path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [7]:
df

,user_id,product_id,rating
0,541312140,44600062,0.018381
1,541312140,44600062,0.018381
2,520558810,44600062,0.018381
3,520558810,44600062,0.018381
4,546203331,44600062,0.018381
...,...,...,...
42448759,563587006,49300114,0.020300
42448760,515624552,46800037,0.017300
42448761,515624552,46800037,0.017300
42448762,515474976,19002087,0.017300
